In [10]:
from pathlib import Path
from collections import Counter
import shutil
import partitura as pt

dataset_path = Path("../data/jazzmus_dataset/")

In [11]:
print("Number of pieces in the dataset:")
len(list(dataset_path.glob("*.json")))

Number of pieces in the dataset:


293

In [12]:
songs = []
for piece_path in dataset_path.glob("*.json"):
    piece_name = piece_path.stem
    if "version" in piece_name:
        piece_name = piece_name.split("_version")[0]
    songs.append(piece_name)

unique_songs = Counter(songs)
print("Number of unique pieces in the dataset:", len(unique_songs))
# unique_songs

Number of unique pieces in the dataset: 163


In [13]:
unique_songs

Counter({'a-day-in-the-life-of-a-fool': 2,
         'a-fella-with-an-umbrella': 1,
         'a-night-in-tunisia': 1,
         'a-pretty-girl-is-like-a-melody': 1,
         'ain-t-misbehavin': 13,
         'alexanders-ragtime-band': 1,
         'all-alone': 1,
         'always': 1,
         'anouman': 1,
         'artillerie-lourde': 1,
         'ask-me-now': 1,
         'ba-lue-bolivar-ba-lues-are': 1,
         'belleville': 1,
         'bemsha-swing': 1,
         'bidin-my-time': 1,
         'bim-bom': 1,
         'bird-food': 1,
         'birk-s-works': 2,
         'black-and-tan-fantasy': 1,
         'blue-haze': 1,
         'blue-monk': 1,
         'blue-skies': 3,
         'blue-train': 1,
         'boplicity': 1,
         'brilliant-corners': 1,
         'by-strauss': 2,
         'c-jam-blues': 1,
         'cabaret': 1,
         'caravan': 1,
         'central-park-west': 1,
         'charade': 1,
         'clap-yo-hands': 1,
         'come-sunday': 1,
         'con-alma': 1,
   

# Dataset problems

## Weird chords

In [75]:
all_chords = []
all_pieces = []
symbol_to_ignore = [".","=","*","**mxhm","*-", "==", "=:|!", "=||", "=!|:", "!", "=:|!|:" , "=||"]

songs = [s for s in dataset_path.glob("*.krn")]
print("Number of unique pieces in the dataset:", len(unique_songs))
# unique_songs

for piece_path in songs:
    # the kern file is made of 2 tab separated columns, read only the second column
    with open(piece_path, "r") as f:
        # read the second column
        for line in f:
            columns = line.strip().split("\t")
            if len(columns) ==2 :
                # if columns[1]== "7" or columns[1] == "-7":
                #     print("!!!!Warning",piece_path)
                #     continue
                # if "other" in columns[1]:
                #     print("!!!!Strange other",piece_path)
                #     continue
                # if "none" in columns[1]:
                #     print("!!!!Strange none",piece_path)
                if columns[1] not in symbol_to_ignore:
                    all_chords.append(columns[1])
                    all_pieces.append(piece_path.stem)

# Counter(all_chords)

Number of unique pieces in the dataset: 163


In [78]:
# split root and chord type
roots = [c.split(" ")[0] for c in all_chords]
chord_types = []
chord_types_piece = []
for c, p in zip(all_chords, all_pieces):
    if len(c.split(" "))==1: # chord without any extension
        chord_types.append("BAD no type")
    else:
        chord_types.append(c.split(" ")[1] if "/" not in c else c.split(" ")[1].split("/")[0])
    chord_types_piece.append(p)
# chord_types = [c.split(" ")[1] if "/" not in c else c.split(" ")[1].split("/")[0]  for c in all_chords]
bass = [c.split("/")[1] if "/" in c else "none" for c in all_chords]

print("roots", Counter(roots))
print("type", Counter(chord_types))
print("bass", Counter(bass))


roots Counter({'G': 1870, 'C': 1637, 'D': 1387, 'F': 1244, 'B-': 1122, 'A': 972, 'E-': 820, 'E': 681, 'B': 569, 'A-': 372, 'D-': 228, 'F#': 180, 'C#': 129, 'G#': 120, 'G-': 64, 'C-': 11, 'D#': 2, 'A#': 2, 'F-': 1, '-7': 1, '7': 1})
type Counter({'dominant': 3670, 'major': 2080, 'minor-seventh': 1643, 'minor': 972, 'major-seventh': 745, 'diminished': 387, 'dominant-ninth': 333, 'half-diminished': 293, 'major-sixth': 238, 'minor-sixth': 200, 'dominant-seventh': 135, 'augmented-seventh': 123, 'augmented': 117, 'diminished-seventh': 100, 'dominant-13th': 93, 'suspended-fourth': 74, 'other': 71, 'none': 55, 'minor-ninth': 35, 'minor-11th': 29, 'major-ninth': 6, 'dominant-11th': 4, 'BAD no type': 4, 'maj69': 2, 'major-minor': 2, '7b9': 1, 'dim7': 1})
bass Counter({'none': 10937, 'G': 85, 'F': 68, 'A': 58, 'E-': 51, 'B-': 46, 'D': 37, 'F#': 35, 'B': 29, 'C': 22, 'E': 18, 'D#': 13, 'D-': 8, 'A-': 5, 'G-': 1})


In [85]:
# problem 1: pieces with -7 and 7 as root
print("Pieces with -7 and 7 as root")
bad_pieces = []
for c, p in zip(roots, all_pieces):
    if c == "-7" or c == "7":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()

# problem 2: pieces with "other" as chord type
print("Pieces with 'other' as chord type")
bad_pieces = []
for c, p in zip(chord_types, chord_types_piece):
    if c == "other":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()

# problem 3: pieces with "none" as chord type
print("Pieces with 'none' as chord type")
bad_pieces = []
for c, p in zip(chord_types, chord_types_piece):
    if c == "none":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()

# problem 4: pieces with "BAD no type" as chord type
print("Pieces with no chord type (it should be major)")
bad_pieces = []
for c, p in zip(chord_types, chord_types_piece):
    if c == "BAD no type":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()

# problem 5: pieces with "dim7" as chord type
print("Pieces with 'dim7' as chord type")
bad_pieces = []
for c, p in zip(chord_types, chord_types_piece):
    if c == "dim7":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()


# problem 6: pieces with "maj69" as chord type
print("Pieces with 'maj69' as chord type")
bad_pieces = []
for c, p in zip(chord_types, chord_types_piece):
    if c == "maj69":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()

# problem 7: pieces with "7b9" as chord type
print("Pieces with '7b9' as chord type")
bad_pieces = []
for c, p in zip(chord_types, chord_types_piece):
    if c == "7b9":
        bad_pieces.append(p)
print(Counter(bad_pieces))
print()

Pieces with -7 and 7 as root
Counter({'they-say-it-s-wonderful_version_1': 1, 'who-cares': 1})

Pieces with 'other' as chord type
Counter({'folie-a-amphion': 6, 'ain-t-misbehavin_version_1': 5, 'ain-t-misbehavin_version_10': 5, 'ain-t-misbehavin_version_11': 5, 'ain-t-misbehavin_version_12': 5, 'ain-t-misbehavin_version_13': 5, 'ain-t-misbehavin_version_4': 5, 'ain-t-misbehavin_version_5': 5, 'ain-t-misbehavin_version_6': 5, 'ain-t-misbehavin_version_7': 5, 'ain-t-misbehavin_version_8': 5, 'ain-t-misbehavin_version_9': 5, 'embraceable-you_version_10': 3, 'i-was-doing-all-right_version_2': 3, 'they-say-it-s-wonderful_version_2': 3, 'the-best-things-happen-while-you-re-dancing_version_2': 1})

Pieces with 'none' as chord type
Counter({'blue-skies_version_2': 20, 'blue-skies_version_1': 17, 'blue-skies_version_3': 16, 'embraceable-you_version_1': 1, 'i-was-doing-all-right_version_1': 1})

Pieces with no chord type (it should be major)
Counter({'embraceable-you_version_22': 1, 'say-it-isn-

Good classes:
- Major 7: major, major-seventh, major-sixth, major-ninth, 
- Minor 7 : minor, minor-seventh, minor-sixth, minor-ninth, minor-11th
- Dominant: dominant, dominant-seventh, dominant-ninth, dominant-11th, dominant-13th, augmented, augmented-seventh, 7b9
- Half-diminished: half-diminished
- Diminished: diminished, diminished-seventh
- Sus: suspended-fourth
- min-maj: major-minor


Bad classes: 
- none (55 occurrencies),
- other (71 occurrencies)
- chords with no type (4 occurrencies). If there is no type it should be "major"
- dim7 (1 occurrencies). I believe this should be substituted with diminished-seventh.
- maj69 (2 occurrencies). This is not necessarily a bad label, but it is how we want to call it? 
- 7b9 . Same as before, this is not necessarily bad, but it is as we want to call it?
- what about other dominant alterated chords, such as 7#9 or 7b13 ? They are not currently in the dataset and it is ok, but how would we call them if they were there?

## Dataset statistics

### Different chords

In [ ]:
all_chords = []
all_pieces = []
symbol_to_ignore = [".","=","*","**mxhm","*-", "==", "=:|!", "=||", "=!|:", "!", "=:|!|:" , "=||"]

unique_songs = [s for s in dataset_path.glob("*.krn") if ("version" not in s.name or "version_1.krn" in s.name)]
print("Number of unique pieces in the dataset:", len(unique_songs))
# unique_songs

for piece_path in unique_songs:
    # the kern file is made of 2 tab separated columns, read only the second column
    with open(piece_path, "r") as f:
        # read the second column
        for line in f:
            columns = line.strip().split("\t")
            if len(columns) ==2 :
                if columns[1]== "7" or columns[1] == "-7":
                    print("!!!!Warning",piece_path)
                    continue
                if "other" in columns[1]:
                    print("!!!!Strange other",piece_path)
                    continue
                if "none" in columns[1]:
                    print("!!!!Strange none",piece_path)
                if columns[1] not in symbol_to_ignore:
                    all_chords.append(columns[1])
                    all_pieces.append(piece_path.stem)

Counter(all_chords).most_common(10)

Number of unique pieces in the dataset: 163


[('C:7', 267),
 ('F:7', 258),
 ('G:7', 236),
 ('Bb:7', 221),
 ('D:7', 183),
 ('F:maj', 177),
 ('C:maj', 157),
 ('Bb:maj', 149),
 ('G:min7', 149),
 ('C:min7', 145)]

In [ ]:
Counter(all_chords)

In [ ]:
# split root and chord type
roots = [c.split(":")[0] for c in all_chords]
bass = [c.split("/")[1] if "/" in c else "none" for c in all_chords]
chord_types = []
for c in all_chords:
    if len(c.split(":"))==1: # chord without any extension
        chord_types.append("none")
    else:
        chord_types.append(c.split(":")[1] if "/" not in c else c.split(":")[1].split("/")[0])
# elements between parentheses in chord_types are extensions modifiers
extensions_full = []
for c in chord_types:
    if "(" in c:
        extensions_full.append(c.split("(")[1].split(")")[0])
    else:
        extensions_full.append("none")
extensions_single = []
for ext in extensions_full:
    extensions_single.extend(ext.split(","))
# now remove extensions from chord_types
chord_types = [c.split("(")[0] if "(" in c else c for c in chord_types]



print(Counter(bass))
print(Counter(chord_types))
print(Counter(roots))

Counter({'none': 5602, 'F': 49, 'G': 47, 'Eb': 44, 'Bb': 34, 'A': 26, 'C': 22, 'D': 22, 'B': 22, 'E': 18, 'F#': 12, 'Ab': 5, 'Db': 5, 'D#': 4, 'Gb': 1})
Counter({'7': 1734, 'min7': 1007, 'maj': 915, 'maj7': 459, 'min': 457, '9': 184, 'maj6': 172, 'hdim7': 164, 'dim': 125, '7(b9)': 112, '7(b5)': 77, 'aug7': 74, 'min6': 59, 'dim7': 56, '7(#9)': 55, '13': 46, 'aug': 32, 'min9': 28, 'none': 21, 'sus4(7)': 14, 'min11': 12, '9(b5)': 11, '7(#11)': 11, 'sus4(7,9)': 11, 'maj6(9)': 10, '13(b9)': 9, 'min(#5)': 8, '7(#5)': 6, '11': 4, '7(#5,b9)': 4, '7(b5,#9)': 4, 'maj9': 4, 'maj7(#5)': 4, 'maj7(#11)': 3, '9(#11)': 3, 'maj6(b9)': 2, '13(#11)': 2, 'aug(6,9)': 2, 'minmaj7': 2, 'aug7(#9)': 2, 'sus4': 2, '7(4,3)': 1, '7(#7)': 1, 'min7(#5)': 1, '7(5,b9,#9,#11,b13)': 1, 'min6(9)': 1, 'sus4(7,13)': 1})
Counter({'C': 915, 'F': 866, 'Bb': 745, 'G': 741, 'D': 558, 'Eb': 549, 'A': 474, 'E': 285, 'Ab': 255, 'B': 192, 'Db': 157, 'F#': 58, 'Gb': 50, 'C#': 39, 'G#': 18, 'Cb': 7, 'D#': 2, 'Fb': 1, 'A#': 1})


### MusicXML chords

In [5]:
all_chords = []
symbol_to_ignore = [".","=","*","**mxhm","*-", "==", "=:|!", "=||", "=!|:", "!", "=:|!|:" , "=||"]

unique_songs = [s for s in dataset_path.glob("*.musicxml") if ("version" not in s.name or "version_1.krn" in s.name)]
print("Number of unique pieces in the dataset:", len(unique_songs))
# unique_songs

for piece_path in unique_songs:
    # the kern file is made of 2 tab separated columns, read only the second column
    with open(piece_path, "r") as f:
        # read the second column
        for line in f:
            columns = line.strip().split("\t")
            if len(columns) ==2 :
                if columns[1]== "7" or columns[1] == "-7":
                    print("!!!!Warning",piece_path)
                    continue
                if "other" in columns[1]:
                    print("!!!!Strange other",piece_path)
                    continue
                if "none" in columns[1]:
                    print("!!!!Strange none",piece_path)
                if columns[1] not in symbol_to_ignore:
                    all_chords.append(columns[1])

Counter(all_chords).most_common(10)

Number of unique pieces in the dataset: 0


[]

In [61]:
Counter(all_chords)

Counter({'A minor': 97,
         'B half-diminished': 33,
         'E dominant': 110,
         'D minor-seventh': 137,
         'G dominant': 258,
         'C major': 157,
         'E half-diminished': 18,
         'A dominant': 143,
         'C major-seventh': 41,
         'F major-seventh': 74,
         'D dominant': 192,
         'D minor': 91,
         'A minor/G': 1,
         'A minor-seventh': 144,
         'E minor-seventh': 57,
         'A minor-sixth': 8,
         'A- major': 37,
         'B- minor': 19,
         'E- major': 133,
         'F minor': 39,
         'E- dominant': 140,
         'A half-diminished': 14,
         'G minor': 68,
         'G half-diminished': 19,
         'C dominant': 303,
         'F major-sixth': 32,
         'C minor-seventh': 145,
         'E major': 10,
         'F major': 177,
         'F dominant': 275,
         'B- major': 149,
         'G minor-seventh': 149,
         'E- major-seventh': 80,
         'E other': 2,
         'F minor-seventh':